# DSP Filter Design

## Required Packages

Must add "DSP" package:
```
Pkg.add("DSP")
```

In [ ]:
using DSP
include("DSPFilters.jl")
using InspectDR
using NumericIO
using Colors

#Convenience aliases:
SI(x) = formatted(x, :SI, ndigits=3)

#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
#(SVG outputs do not render well in notebooks for some reason...)
InspectDR.defaults.rendersvg = false
;

# Setup: Interact, Reactive, and Signals
(Also create initial plot object)

In [ ]:
using Interact, Reactive

ftypelist = ["Highpass", "Lowpass", "Bandstop", "Bandpass"]
fimpllist = ["Butterworth", "Elliptic", "ChebyshevPass", "ChebyshevStop"]

ftype = Signal(String, "Lowpass") #Filter type
fimpl = Signal(String, "Elliptic") #Filter implementation
fmax = Signal(Float64, 500) #Maximum plot frequency
forder = Signal(Int, 6) #Filter order
fl = Signal(Float64, 300) #Low frequency point
fh = Signal(Float64, 400) #High frequency point
rpass = Signal(Float64, 5) #Pass-band ripple
rstop = Signal(Float64, 40) #Stop-band ripple
pobj = DSPFilters.newplot() #Create initial plot object
;

# Interact/Reactive Control: Plot limits


## TODO

In [ ]:
txt_fmax = textbox(value(fmax), label="max freq") #This one does not cause hangups
fmax = signal(txt_fmax) #Hack to get fmax/txt_fmax working
map(display, [txt_fmax])
;

# Interact/Reactive Control: Inline Plots
Use sliders/text boxes to change filter parameters.

## TODO

 - Make fl & fh sliders relative to fmax?
 - Disable unused parameters (depending on filter implementation)
 - Look for off by 1 errors (esp. for FFT vector lengths, etc).
 - Other details look not quite right (@ notches, phase response, ...)

In [ ]:
tgl_ftype = togglebuttons(ftypelist, value_label=value(ftype), signal=ftype)
tgl_fimpl = dropdown(fimpllist, value_label=value(fimpl), signal=fimpl)
map(display, [tgl_ftype, tgl_fimpl])

#Main filter parameters:
sld_order = slider(2:1:30, value=value(forder), label="Filter order", signal = forder)
sld_rpass = slider(0.1:.1:20, value=value(rpass), label="Pass-band ripple (dB)", signal = rpass)
sld_rstop = slider(5:.1:200, value=value(rstop), label="Stop-band ripple (dB)", signal = rstop)
sld_fl = slider(10:10:1e3, value=value(fl), label="Low frequency (Hz)", signal = fl)
sld_fh = slider(10:10:1e3, value=value(fh), label="High frequency (Hz)", signal = fh)
map(display, [sld_order, sld_rpass, sld_rstop, sld_fl, sld_fh])

#Widget-controlled plot:
updater_plot = map(fmax, ftype, fimpl, forder, fl, fh, rpass, rstop) do _fmax, _ftype, _fimpl, _forder, _fl, _fh, _rpass, _rstop
    _filttypeid = Symbol(lowercase(_ftype))
    _filtimplid = Symbol(lowercase(_fimpl))
    DSPFilters.update(pobj, _fmax, _filttypeid, _filtimplid, _forder, _fl, _fh, _rpass, _rstop)
end
display(updater_plot)
;

# Interact/Reactive Control: Inspect/Gtk GUI

In [ ]:
gtkgui = display(InspectDR.GtkDisplay(), pobj) #Display plot in Gtk GUI.

#Refresh GUI when plot changes:
updater_gtkgui = map((p)->InspectDR.refresh(gtkgui), updater_plot)

#Re-display GUI controls, for convenience:
#map(display, widgetlist)
;

# Demo Complete!